# Solving problems by Searching

This notebook serves as supporting material for topics covered in **Chapter 3 - Solving Problems by Searching** and **Chapter 4 - Beyond Classical Search** from the book *Artificial Intelligence: A Modern Approach.* This notebook uses implementations from [search.py](https://github.com/aimacode/aima-python/blob/master/search.py) module. Let's start by importing everything from search module.

In [268]:

import random
import bisect
# Needed to hide warnings in the matplotlib sections
import warnings
import math
import statistics
import pandas as pd
import time
import sys,os
# from numpy import *
warnings.filterwarnings("ignore")
current_timer =0
fitnesses1 = []

For visualisations, we use networkx and matplotlib to show the map in the notebook and we use ipywidgets to interact with the map to see how the searching algorithm works. These are imported as required in `notebook.py`.

## CONTENTS

* Genetic Algorithm


## GENETIC ALGORITHM

Genetic algorithms (or GA) are inspired by natural evolution and are particularly useful in optimization and search problems with large state spaces.

Given a problem, algorithms in the domain make use of a *population* of solutions (also called *states*), where each solution/state represents a feasible solution. At each iteration (often called *generation*), the population gets updated using methods inspired by biology and evolution, like *crossover*, *mutation* and *natural selection*.

PERMUTATION ENCODING

GA Parameters:
We now need to define the maximum size of each population. Larger populations have more variation but are computationally more  expensive to run algorithms on.
As our population is not very large, we can afford to keep a relatively large mutation rate.
Termination after a predefined number of generations.
N is the size of the chromosmes, and [0,..,N-1] is the alphabet

Great! Now, we need to define the most important metric for the genetic algorithm, i.e the fitness function. This will simply return the number of matching characters between the generated sample and the target phrase.

In [269]:
def fitness_fn(sample):
    return fitness_v2(sample)
    
# target
def fitness_v1(sample):
    # initialize fitness to 0
    fitness = 0
    for i in range(len(sample)):
        # increment fitness by 1 for every matching character
        if sample[i] == target[i]:
           fitness += 1
    return fitness

def cost(sample):
    # Asegura que el cromosoma forme un circuito completo (ciudad 0 al principio y al final)
    chromosome_with_start_city = [0] + sample + [0]
    
    # Calcula la distancia total recorrida por el cromosoma
    total_distance = 0
    for i in range(len(chromosome_with_start_city) - 1):
        # Obtiene el índice de las ciudades en el grafo
        city1 = chromosome_with_start_city[i]
        city2 = chromosome_with_start_city[i + 1]
        # Suma la distancia entre la ciudad 1 y la ciudad 2
        total_distance += grafo.Dist[city1][city2]
    # La distancia total (queremos minimizar la distancia)
    return total_distance

def fitness_v2(sample):
    # El fitness es el inverso de la distancia total (queremos minimizar la distancia)
    fitness = 3 ** ( 10000 / cost(sample) )
    return fitness

To generate `ngen` number of generations, we run a `for` loop `ngen` number of times. After each generation, we calculate the fitness of the best individual of the generation and compare it to the value of `f_thres` using the `fitness_threshold` function. After every generation, we print out the best individual of the generation and the corresponding fitness value. Lets now write a function to do this.

In [270]:
current_timer=0
# Save results to excel file by passing the name of the sheet and the data
# This will append the new date to a new sheet in the excel file
def save_to_excel(curren_best, generation, best_fitness, average_fitness, stDev, sheet):
    output_file_path = "results_genetics.xlsx"
    df = pd.DataFrame([[curren_best, generation, best_fitness, average_fitness, stDev]], columns=['Current best', 'Generation', 'Best Fitness', 'Average Fitness', 'Standard Dev'])
    # Check if the file exists
    if not os.path.exists(output_file_path):
        # If the file does not exist, create it and save the DataFrame to it
        df.to_excel(output_file_path, sheet_name=sheet, index=False)
        print(f"File '{output_file_path}' created and data saved.")
    else:
        # If the file exists, append the DataFrame to the existing file
        data1 = pd.read_excel(output_file_path, sheet_name=sheet)
        df = pd.concat([df, data1], axis=0, ignore_index=True)
        with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, index=False, sheet_name=sheet)
        print(f"Data appended to '{output_file_path}'.")

def genetic_algorithm_stepwise(population, fitness_fn, gene_pool, ngen=1200, pmut=0.05):
   
    for generation in range(int(ngen)):
        result ={"average":[], "max":[]}
        elapsed = time.time()
        # Elitism may be here - ADDED
        previous_best = max(population, key=fitness_fn)
        population = [mutate(uniform_crossover(*select(2, population, fitness_fn)), pmut) for i in range(len(population)-1)]
        population.append(previous_best)
        # stores the individual genome with the highest fitness in the current population
        current_best = max(population, key=fitness_fn)
        fitnesses = list(map(fitness_fn, population))
        elapsed = time.time()-elapsed
        if len(fitnesses1)>generation:
            result = fitnesses1[generation]
            
        result["max"].append(max(fitnesses))
        result["average"].append(statistics.mean(fitnesses))
        if len(fitnesses1)>generation:
            fitnesses1[generation] = result
        else:
            fitnesses1.append(result)
            
        print(f'Current best: {current_best}\tGeneration: {generation:04d}   BestFitness: {max(fitnesses):10f}\tAverageFitness: {statistics.mean(fitnesses):10f}\tStDev: {statistics.stdev(fitnesses):10f}\r')
       
        
        
    return max(population, key=fitness_fn)       

def init_population(pop_number, gene_pool):
    # a chromosome is a random permutation of the alphabet
    population = []
    for _ in range(pop_number):
        # Shuffle the gene pool and take the first pool_size elements as an individual
        v = gene_pool[:]
        random.shuffle(v)
        population.append(v)
    return population

def select(r, population, fitness_fn):
    fitnesses = map(fitness_fn, population)
    #scaling here
    sampler = weighted_sampler(population, fitnesses)
    ret = [sampler() for i in range(r)]
    return ret

def weighted_sampler(seq, weights):
    """Return a random-sample function that picks from seq weighted by weights."""
    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)
    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]
    # bisect(a,x) -> insertion position of a in a sorted list x - AL REVES

def uniform_crossover(x, y):
    # x, y permutations of the alphabet
    n = 0
    child = [-1] * (N-1)
    indexes = [0] * (N-1)
    # de x se copian los valores de las posiciones con indexex[i] == 1 en las mismas posiciones en child
    for i in  range((N-1)):
        indexes[i] = random.randint(0,1) 
        if indexes[i] == 1:
            child[i] = x[i]
            n += 1
    # El resto (N-n) se copia de y en su orden relativo, desde el principio
    i = 0 # indice en y
    k = 0 # indice en child
    for t in range(N-n-1):
        while y[i] in child[:]:
            i += 1
        while child[k] != -1:
            k += 1
        child[k] = y[i]
        i += 1   
    return child

def order_crossover(x, y): # OX
    # TODO implementar cruce en dos puntos
    ycop = y.copy()
    
    p1 = random.randrange(0, N-1)
    p2 = random.randrange(0, N-1)
    while p2 == p1:
        p2 = random.randrange(0, N-1)
    
    child = [-1] * (N-1)
    
    i = p1
    while i != p2:
        child[i] = x[i]
        if x[i] in ycop:
            ycop.remove(x[i])
        i = ( i + 1 ) % (N-1)
    
    counter = 0
    while i != p1:
        child[i] = ycop[counter]
        counter += 1
        i = ( i + 1 ) % (N-1)
    
    return child

def mutate(x, pmut):
    # if random.uniform(0, 1) > 0.5:
    #     return random_mutate(x, pmut)
    return adyacency_mutate(x, pmut)

# Random mutation
def random_mutate(x, pmut):
    if random.uniform(0, 1) >= pmut:
        return x
    i, j = random.sample(range(int(N)-1), 2)
    x[i], x[j] = x[j], x[i]
    return x

# DONE Mutation method that exchanges two contiguous genes
def adyacency_mutate(x, pmut):
    if random.uniform(0, 1) >= pmut:
        return x
    i = random.randrange(0, N-1)
    j = (i + 1) % (N-1) 
    x[i], x[j] = x[j], x[i]
    return x


The function defined above is essentially the same as the one defined in `search.py` with the added functionality of printing out the data of each generation.

In [271]:
class Grafo:
    """Un Grafo es un array bidimensional de tamño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (i,j) es la distancia entre las ciudades i y j,
     que es el mismo que la distancia entre j e i """
    def __init__(self,lista):
        "N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis EDGE_WEIGHT_TYPE"
        self.N = int(((8*len(lista)+1)**0.5)-1)/2 # NUMERO DE CIUDADES, 0 ES LA CIUDAD DE PARTIDA
        #print("Número de ciudades: ",self.N)
        self.ciudades = list(range(int(self.N)))
        #print("Ciudades: ", self.ciudades)
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        #print("Matriz de costes (Dist) ANTES:")
        #self.printDist(self.Dist, self.N)
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y] # EL VALOR DE LA POSICION (X,Y) ES LA DISTANCIA ENTRE LAS CIUDADES X E Y
        #print("Matriz de costes (Dist) DESPUES:")
        #self.printDist(self.Dist, self.N)

    def printDist(self, dist, N):
        for i in range(int(N)):
            print("[", end="")
            for j in range(int(N)):
                print(dist[i][j], end=" ")
            print("]", end="")
            print()

We have defined all the required functions and variables. Let's now create a new population and test the function we wrote above.

In [272]:
gr21 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,

]
gr18 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
]

gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]

gr12 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
 ]



# creamos el grafo
# no se permite el gene 0 (han de ser de 1 a N-1)
grafo = Grafo(gr12)

# max_population = 100
max_population = 200
mutation_rate = 0.1 # 7% of the chromosones are mutated
ngen = 200 # maximum number of generations

N = int(grafo.N) # chromosome size
gene_pool = list(range(1, N)) # alphabet
target = [15, 11, 8, 4, 1, 9, 10, 2, 14, 13, 16, 5, 7, 6, 12, 3] # used for basic fitness function
print(gene_pool)
print(target)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[15, 11, 8, 4, 1, 9, 10, 2, 14, 13, 16, 5, 7, 6, 12, 3]


In [273]:
def save_to_excelt(target, solution, fitness, time, cost, sheet, iteration, length):
    output_file_path = "results_genetics_times.xlsx"
    df = pd.DataFrame([[target, solution, fitness, time, cost,iteration, length]], columns=['Target', 'Solution', 'Fitness', 'Tiempo', 'Coste',  'Iteracion','Longitud'])
    # Check if the file exists
    if not os.path.exists(output_file_path):
        #pd.ExcelWriter(output_file_path, engine='openpyxl',mode='a', if_sheet_exists='overlay')
        # If the file does not exist, create it and save the DataFrame to it
        df.to_excel(output_file_path, sheet_name='ox', index=False)
        print(f"File '{output_file_path}' created and data saved.")
    else:
        # If the file exists, append the DataFrame to the existing file
        data1 = pd.read_excel(output_file_path, sheet_name='ox')
        df = pd.concat([df, data1], axis=0, ignore_index=True)
        with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, index=False, sheet_name='ox')
        print(f"Data appended to '{output_file_path}'.")

def save_to_excel_fitness_row(generation, maxim, average,cruce, poblacion):
    output_file_path = "results_genetics_generations.xlsx"
    df = pd.DataFrame([[generation, maxim, average, cruce, poblacion]], columns=['Generation', 'Max_Fitness', 'Avg_Fitness', 'Tipo_cruce','Poblacion'])
    # Check if the file exists
    if not os.path.exists(output_file_path):
        #pd.ExcelWriter(output_file_path, engine='openpyxl',mode='a', if_sheet_exists='overlay')
        # If the file does not exist, create it and save the DataFrame to it
        df.to_excel(output_file_path, index=False)
        print(f"File '{output_file_path}' created and data saved.")
    else:
        # If the file exists, append the DataFrame to the existing file
        data1 = pd.read_excel(output_file_path)
        df = pd.concat([df, data1], axis=0, ignore_index=True)
        with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, index=False)
        print(f"Data appended to '{output_file_path}'.")


def save_to_excelfitness(cruce, poblacion):

    for i in range(len(fitnesses1)):
        save_to_excel_fitness_row(i,statistics.mean(fitnesses1[i]["max"]),statistics.mean(fitnesses1[i]["average"]), cruce, poblacion)






   
population = init_population(max_population, gene_pool)
# print(population)
# random.seed(10) # set seed to debug
for i in range(30):
    seconds = time.time()
    solution = genetic_algorithm_stepwise(population, fitness_fn, gene_pool, ngen, mutation_rate)
    elapsed = time.time()-seconds
    print("Time:")
    print(elapsed)
    print("Target: ")
    print(target)
    print("Solution: ")
    print(solution)
    print("Fitness: ")
    print(fitness_fn(solution))
    print("Cost:")
    print(cost(solution))
    #Escribir tiempo total Tiempo total
    save_to_excelt(target, solution, fitness_fn(solution), elapsed,cost(solution), 'ux',i,N)
save_to_excelfitness('ux',N)


Current best: [6, 3, 2, 4, 5, 7, 10, 9, 1, 8, 11]	Generation: 0000   BestFitness:  77.953722	AverageFitness:  26.359177	StDev:  11.337671
Current best: [6, 3, 7, 5, 2, 1, 9, 10, 4, 11, 8]	Generation: 0001   BestFitness: 198.228185	AverageFitness:  30.304263	StDev:  18.707635
Current best: [6, 3, 7, 5, 2, 1, 9, 10, 4, 11, 8]	Generation: 0002   BestFitness: 198.228185	AverageFitness:  32.519068	StDev:  21.352018
Current best: [6, 3, 7, 5, 2, 1, 9, 10, 4, 11, 8]	Generation: 0003   BestFitness: 198.228185	AverageFitness:  34.156830	StDev:  22.602243
Current best: [6, 3, 7, 5, 2, 1, 9, 10, 4, 8, 11]	Generation: 0004   BestFitness: 207.609620	AverageFitness:  38.629712	StDev:  31.471642
Current best: [6, 3, 7, 5, 2, 1, 9, 10, 4, 8, 11]	Generation: 0005   BestFitness: 207.609620	AverageFitness:  42.784980	StDev:  35.286114
Current best: [3, 6, 7, 5, 2, 1, 9, 10, 4, 8, 11]	Generation: 0006   BestFitness: 247.049869	AverageFitness:  58.498479	StDev:  49.683607
Current best: [3, 6, 7, 5, 2, 1, 9

ValueError: Worksheet named 'ux' not found

## Lo que hay que incluir en el documento
- Avg. Tardiness (fitness) vs time (minutes/generations) - Indica la convergencia del algoritmo genético
    - Para una única ejecución
        - Debe incluir el mejor individuo de cada generación
        - Y el AVG fitness de la generación
    - Para conclusiones, necesitas n > 30 ejecuciones
        - Promedio de mejores individuos
        - Promedio de promedios
        
    - Comparar UX con OX
    - No sirve mucho comparar tamaño de generacion u otros tipos de elitismo
    - Añadir info añadida: grafo (gr17), población (N)...
    - Obligatorio: Hacer otra gráfica similar para gr21: también con UX e OX
        - Plus: añadir esto para una instancia similar, como **Burma52** o similar
    
    Cuando comparas con A*, que sabemos el coste óptimo
        Así, puedes utilizar la gráfica para saber cuánto puedes tardar en sacar el óptimo en A* vs genético

- Título: Que sea informativo y hable del TSP, no sólo "práctica"
- Palabras clave: no poner más de lo que hay
- 4- Aplicación:
    - Dos secciones, uno A* y otro genéticos
    - Explicar muy bien el espacio de b´suqueda
    - Explicar los heurísticos, **especialmente el h3**
- 5- Estudio experimental:
    - **Lo más valorado**
    - La organización de subsecciones es más libre, especialmente dentro de los resultados experimentales
    - En la seccion de diseño
        - Hablar de que instancias, y qué número de ejecuciones
        - Y de otras variables importantes como ratio de mutación
    - En resultados
        - Es importante comparar tanto lo informados que están los heurísticos como el tiempo que tardan de forma práctica
        - Notar qué heurístico es el mejor, (depende del número de instancias?)
        - Hacer comparación de A* con PEA* (Weighted A* / epsillon A*)
            - Menos nodos expandidos, pero peor solución
            - Comprobar que el coste no supera (1+e)C*
            - En todas las instancias (una tabla)
        - Algoritmos genéticos
            - Convergencia (leer arriba)
            - Probar otro escalado del fitness
            - U otras mutaciones
        - Conclusión con comparativa entre genéticos y A*
- Conclusión:
    - qué algoritmo es el mejor para qué casos de uso
        - En casod de ser A*, qué heurístico y por qué
        - En caso de ser genético, qué cruce/mutación y por qué
    

Hay que tener un Excel con los datos de los experimentos publicados

Usar numpy para generar las data tables

The genetic algorithm was able to converge!
We implore you to rerun the above cell and play around with `target, max_population, f_thres, ngen` etc parameters to get a better intuition of how the algorithm works. To summarize, if we can define the problem states in simple array format and if we can create a fitness function to gauge how good or bad our approximate solutions are, there is a high chance that we can get a satisfactory solution using a genetic algorithm. 
- There is also a better GUI version of this program `genetic_algorithm_example.py` in the GUI folder for you to play around with.

#### Eight Queens

Let's take a look at a more complicated problem.

In the *Eight Queens* problem, we are tasked with placing eight queens on an 8x8 chessboard without any queen threatening the others (aka queens should not be in the same row, column or diagonal). In its general form the problem is defined as placing *N* queens in an NxN chessboard without any conflicts.

First we need to think about the representation of each solution. We can go the naive route of representing the whole chessboard with the queens' placements on it. That is definitely one way to go about it, but for the purpose of this tutorial we will do something different. We have eight queens, so we will have a gene for each of them. The gene pool will be numbers from 0 to 7, for the different columns. The *position* of the gene in the state will denote the row the particular queen is placed in.

For example, we can have the state "03304577". Here the first gene with a value of 0 means "the queen at row 0 is placed at column 0", for the second gene "the queen at row 1 is placed at column 3" and so forth.

We now need to think about the fitness function. On the graph coloring problem we counted the valid edges. The same thought process can be applied here. Instead of edges though, we have positioning between queens. If two queens are not threatening each other, we say they are at a "non-attacking" positioning. We can, therefore, count how many such positionings are there.

Let's dive right in and initialize our population:

In [ ]:
population = init_population(100, gene_pool)
print(population[:5])

[[1, 5, 15, 6, 16, 13, 12, 18, 8, 9, 17, 2, 7, 3, 20, 14, 19, 11, 10, 4], [8, 9, 6, 12, 16, 18, 10, 2, 14, 15, 7, 3, 5, 4, 20, 11, 13, 1, 19, 17], [12, 20, 2, 10, 11, 17, 8, 3, 5, 13, 16, 1, 14, 18, 19, 7, 6, 9, 15, 4], [11, 16, 9, 8, 17, 6, 3, 13, 10, 15, 20, 12, 2, 18, 5, 1, 19, 4, 7, 14], [4, 7, 8, 11, 6, 1, 10, 15, 13, 3, 12, 2, 14, 20, 5, 18, 17, 19, 16, 9]]


We have a population of 100 and each individual has 8 genes. The gene pool is the integers from 0 to 7, in string form. Above you can see the first five individuals.

Next we need to write our fitness function. Remember, queens threaten each other if they are at the same row, column or diagonal.

Since positionings are mutual, we must take care not to count them twice. Therefore for each queen, we will only check for conflicts for the queens after her.

A gene's value in an individual `q` denotes the queen's column, and the position of the gene denotes its row. We can check if the aforementioned values between two genes are the same. We also need to check for diagonals. A queen *a* is in the diagonal of another queen, *b*, if the difference of the rows between them is equal to either their difference in columns (for the diagonal on the right of *a*) or equal to the negative difference of their columns (for the left diagonal of *a*). Below is given the fitness function.

In [ ]:
def fitness(q):
    non_attacking = 0
    for row1 in range(len(q)):
        for row2 in range(row1+1, len(q)):
            col1 = int(q[row1])
            col2 = int(q[row2])
            row_diff = row1 - row2
            col_diff = col1 - col2

            if col1 != col2 and row_diff != col_diff and row_diff != -col_diff:
                non_attacking += 1

    return non_attacking

Note that the best score achievable is 28. That is because for each queen we only check for the queens after her. For the first queen we check 7 other queens, for the second queen 6 others and so on. In short, the number of checks we make is the sum 7+6+5+...+1. Which is equal to 7\*(7+1)/2 = 28.

Because it is very hard and will take long to find a perfect solution, we will set the fitness threshold at 25. If we find an individual with a score greater or equal to that, we will halt. Let's see how the genetic algorithm will fare.

In [ ]:
solution = genetic_algorithm_stepwise(population, fitness, gene_pool, ngen=100)
print(solution)
#print(fitness(solution))

Current best: [15, 16, 6, 4, 7, 17, 14, 9, 13, 20, 2, 18, 1, 11, 19, 12, 3, 8, 10, 5]	Generation: 0000   BestFitness: 185.000000	AverageFitness: 177.410000	StDev:   3.539374
Current best: [11, 13, 16, 10, 3, 7, 1, 12, 14, 2, 20, 8, 17, 4, 15, 5, 19, 9, 18, 6]	Generation: 0001   BestFitness: 185.000000	AverageFitness: 177.310000	StDev:   3.509568
Current best: [11, 13, 16, 10, 3, 7, 1, 12, 14, 2, 20, 8, 17, 4, 15, 5, 19, 9, 18, 6]	Generation: 0002   BestFitness: 185.000000	AverageFitness: 176.740000	StDev:   3.823426
Current best: [20, 4, 17, 5, 3, 18, 11, 9, 8, 13, 16, 19, 6, 14, 10, 7, 12, 1, 15, 2]	Generation: 0003   BestFitness: 185.000000	AverageFitness: 176.660000	StDev:   4.045748
Current best: [20, 4, 17, 5, 3, 18, 11, 9, 8, 13, 16, 19, 6, 14, 10, 7, 12, 1, 15, 2]	Generation: 0004   BestFitness: 185.000000	AverageFitness: 177.170000	StDev:   3.829853
Current best: [18, 11, 9, 6, 13, 12, 14, 20, 8, 4, 15, 7, 3, 10, 5, 1, 17, 19, 16, 2]	Generation: 0005   BestFitness: 185.000000	A

Above you can see the solution and its fitness score, which should be no less than 25.

This is where we conclude Genetic Algorithms.

<br>
This concludes the notebook.
Hope you learned something new!